In [26]:
import numpy as np
import torch


class Node:
    def __init__(self, value, func='variable', parents=[], *args):
        self.value = value
        self.func = func
        self.parents = parents
        self.args = args

    def __str__(self):
        return f'({self.value}, {self.func}, {self.args}, {self.parents})'

    def __repr__(self):
        return f'({self.value}, {self.func}, {self.args}, {self.parents})'

In [27]:
def add(a: Node, b: Node):
    # unbox
    va = a.value
    vb = b.value

    # primitive
    vc = np.add(va, vb)

    # box
    c = Node(vc, 'add', [a, b])
    c.args = (va, vb)
    return c

In [28]:
def negative(a: Node):
    # unbox
    va = a.value

    # primitive
    vb = np.negative(va)

    # box
    b = Node(vb, 'negative', [a])
    b.args = (va,)
    return b

In [29]:
def exp(a: Node):
    # unbox
    va = a.value

    # primitive
    vb = np.exp(va)

    # box
    b = Node(vb, 'exp', [a])
    b.args = (va,)
    return b

In [30]:
def reciprocal(a: Node):
    # unbox
    va = a.value

    # primitive
    vb = np.reciprocal(va)

    # box
    b = Node(vb, 'reciprocal', [a])
    b.args = (va,)
    return b

In [31]:
def matmul(a: Node, b: Node):
    va = a.value
    vb = b.value

    vc = va @ vb

    c = Node(vc, 'matmul', [a, b])
    c.args = (va, vb)
    return c

In [67]:
def relu(a: Node):
    va = a.value

    vc = va * (va > 0)

    c = Node(vc, 'relu', [a])
    c.args = (va,)
    return c


a = [1.0, -1.0]
r_a = relu(Node(np.array(a))).value
t_r_a = torch.relu(torch.tensor(a))
np.testing.assert_array_equal(r_a, t_r_a)

In [114]:
def softmax(a: Node):
    va = a.value

    exp_va = np.exp(va)
    vc = exp_va / np.sum(exp_va)

    c = Node(vc, 'softmax', [a])
    c.args = (va,)
    return c


x = [1., 2, 3]
y = softmax(Node(np.array(x)))
t_x = torch.tensor(x, dtype=torch.float64)
t_y = torch.softmax(t_x, dim=-1)
# todo: why not equal?
#np.testing.assert_array_equal(y.value, t_y.cpu().detach().numpy())
assert np.abs(np.sum(y.value - t_y.cpu().detach().numpy())) < 0.0001

In [33]:
def logistic(i):
    return reciprocal(add(Node(1), exp(negative(i))))

In [76]:

def backward_pass(g, end_node, vjps):
    tmp_node = Node(end_node.value, parents=[end_node])
    q = []
    gs = {tmp_node: (g,)}
    q.append(tmp_node)
    while len(q) > 0:
        cur_node = q.pop(0)
        cur_gs = gs[cur_node]
        for node, cur_g in zip(cur_node.parents, cur_gs):
            q.append(node)
            vjp = vjps[node.func]
            grads = vjp(cur_g, node.value, *node.args)
            if node not in gs:
                gs[node] = grads
            else:
                gs[node] += grads
    return gs


In [34]:
def add_vjp(g, ans, a, b):
    return g, g

In [35]:
def exp_vjp(g, ans, a):
    return (ans * g,)

In [36]:
def negative_vjp(g, ans, a):
    return (-1 * g,)

In [37]:
def reciprocal_vjp(g, ans, a):
    return (np.divide(-1, a * a) * g,)

In [38]:
def variable_vjp(g, ans):
    return (g,)

In [48]:
vjps = {
    "add": add_vjp,
    "negative": negative_vjp,
    "exp": exp_vjp,
    "reciprocal": reciprocal_vjp,
    "variable": variable_vjp,
}

In [77]:
def relu_vjp(g, ans, a):
    return (ans * g,)


vjps["relu"] = relu_vjp
a = [1., -1.]
g = [1., 1.]
x = Node(np.array(a))
y = relu(x)
t_x = torch.tensor(a, requires_grad=True)
t_y = torch.relu(t_x)

gs = backward_pass(np.array(g), y, vjps)
t_y.backward(torch.tensor([1., 1.]))

np.testing.assert_array_equal(gs[x][0], t_x.grad)


In [141]:
# why? https://deepnotes.io/softmax-crossentropy
def softmax_vjp(g, ans, a):
    t = ans.reshape(-1, 1)
    jocbian = np.diagflat(t) - np.dot(t, t.T)
    return (jocbian @ g,)


vjps['softmax'] = softmax_vjp
x = [1., 2]
g = [1., 2.]
n_x = Node(np.array(x))
y = softmax(n_x)
t_x = torch.tensor(x, requires_grad=True)
t_y = torch.softmax(t_x, dim=-1)

t_y.backward(torch.tensor(g))
gs = backward_pass(np.array(g), y, vjps)

#np.testing.assert_array_equal(gs[n_x][0], t_x.grad)
assert np.abs(np.sum(gs[n_x][0] - t_x.grad.cpu().detach().numpy())) < 0.0001

In [132]:
a = np.sum([0.09003057, 0.24472847,
            0.66524096])
b = 0.09003057 / a
b - b * b

0.08192506646547511

In [133]:
-0.24472847 * 0.66524096

-0.1628034023221312

In [50]:
def matmul_vjp(g, ans, a, b):
    # a is a matrix
    # b is a vector
    b_d = np.matmul(np.transpose(a), g)
    r, c = a.shape
    # y=Wx
    # dy/dW = [x;x;x] element wise multi g
    a_d = np.multiply(np.tile(b, (r, 1)), g.reshape(r, 1))

    return (a_d, b_d)


vjps["matmul"] = matmul_vjp

In [79]:
import torch

tz = torch.tensor([1.5, 1.5], requires_grad=True)
# reciprocal(add(Node(1), exp(negative(i))))
y = torch.reciprocal(torch.add(1., torch.exp(torch.negative(tz))))
z = Node(np.array([1.5, 1.5]))
logsit = logistic(z)

y.backward(torch.tensor([1., 1.]))
gs = backward_pass((1., 1.), logsit, vjps)

assert np.sum(tz.grad.cpu().detach().numpy() - gs[z]) < 0.00001

In [80]:
t_x = torch.tensor([1.5, 1.6, 1.7], requires_grad=True)
t_w = torch.tensor([
    [1., 2, 3],
    [4, 5, 6]
], requires_grad=True)
t_b = torch.tensor([0.3, 0.4], requires_grad=True)
t_y = torch.matmul(t_w, t_x) + t_b
t_y.backward(torch.tensor([1., 2.]))

x = Node(np.array([1.5, 1.6, 1.7]))
w = Node(np.array([
    [1., 2, 3],
    [4, 5, 6]
]))
b = Node(np.array([0.3, 0.4]))
y = add(matmul(w, x), b)
gs = backward_pass(np.array([1., 2.]), y, vjps)

assert np.sum(t_x.grad.cpu().detach().numpy() - gs[x]) < 0.00001

In [87]:
t_x = torch.tensor([1.5, 1.6, -1700], requires_grad=True)
t_w = torch.tensor([
    [1., 2, 3],
    [4, 5, 6]
], requires_grad=True)
t_b = torch.tensor([0.3, 0.4], requires_grad=True)
t_y = torch.relu(torch.matmul(t_w, t_x) + t_b)
t_y.backward(torch.tensor([1., 2.]))

x = Node(np.array([1.5, 1.6, -1700]))
w = Node(np.array([
    [1., 2, 3],
    [4, 5, 6]
]))
b = Node(np.array([0.3, 0.4]))
y = relu(add(matmul(w, x), b))
gs = backward_pass(np.array([1., 2.]), y, vjps)

np.testing.assert_array_equal(t_x.grad, gs[x][0])

1. single -> batch
2. python -> cuda